In [1]:
import pandas as pd
lexicon = pd.read_csv("../data/acronym_lexicon_partial.csv")

# do a quick filter on the has-tags field
lexicon["has_tags"] = [l!='[]' for l in lexicon["tags"]]
tagged_lexicon = lexicon[lexicon["has_tags"]]

In [2]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

In [3]:
print(len(tagged_lexicon))
len(set(tagged_lexicon["acronym"].to_list()))

129


48

In [4]:
print(len(lexicon))
len(set(lexicon["acronym"].to_list()))

500


61

In [12]:
from typing import List

def zeroshot_classify(text:str, candidate_labels:List[str]= ['finance', 'regulation','government institute','bank','policy','guideline','regulatory body','tax','company']):

    results = classifier(text, candidate_labels,multi_label=True)
    
    return {k:v for k,v in zip(results["labels"], results["scores"])}

def assign_class(text):
    # Do some logic for class assignment
    # if finance <0.4: assign it non-financial-entity
    
    # else
    # if max of non financial tags < 0.4:
    # assign the class as other-financial entity
    # else
    # assign it as the maximum
    
    classification = zeroshot_classify(text)
    
    if classification["finance"] < 0.4:
        output_class = "Non-Financial Entity"
    else:
        del classification["finance"]
        
        scores = sorted(classification.items(), key=lambda x:x[1], reverse=True)

        max_class, max_score = scores[0]
    
        if max_score <0.5:
            output_class = "Other Financial Entity"
            
        else: output_class = max_class
        
    return output_class, classification


In [13]:
# Example usage
#text = tagged_lexicon["summary"].to_list()[0].split(".")
text = "Goverment deparment financial"
classification, data = assign_class(text)
print(classification)
print(data)


regulatory body
{'regulatory body': 0.9633584022521973, 'policy': 0.8513957858085632, 'regulation': 0.5462090373039246, 'bank': 0.3800763785839081, 'guideline': 0.36723998188972473, 'tax': 0.11873148381710052, 'government institute': 0.08829646557569504, 'company': 0.03803764656186104}


In [16]:
from tqdm import tqdm


semantic_type = []
classification_data = []
for summary in tqdm(tagged_lexicon["summary"]):
    classification, data = assign_class(summary.split(".")[0])
    semantic_type.append(classification)
    classification_data.append(data)
    
tagged_lexicon["semantic_type"] = semantic_type
tagged_lexicon["classification_data"] = classification_data


100%|██████████████████████████████████████████████| 129/129 [04:43<00:00,  2.20s/it]
/tmp/ipykernel_331094/73937035.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tagged_lexicon["semantic_type"] = semantic_type
/tmp/ipykernel_331094/73937035.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tagged_lexicon["classification_data"] = classification_data


In [24]:
tagged_lexicon["summary"].to_list()[0]

"The know your customer or know your client (KYC) guidelines in financial services require that professionals make an effort to verify the identity, suitability, and risks involved with maintaining a business relationship. The procedures fit within the broader scope of a bank's anti-money laundering (AML) policy. KYC processes are also employed by companies of all sizes for the purpose of ensuring their proposed customers, agents, consultants, or distributors are anti-bribery compliant, and are actually who they claim to be. Banks, insurers, export creditors, and other financial institutions are increasingly demanding that customers provide detailed due diligence information. Initially, these regulations were imposed only on the financial institutions but now the non-financial industry, fintech, virtual assets dealers, and even non-profit organizations are liable to oblige.\n\n"

In [96]:
# save this lexicon
tagged_lexicon.to_csv("../data/financial_taxonomy.csv")

In [18]:
for acronym in set(tagged_lexicon["acronym"]):
    alex = tagged_lexicon[tagged_lexicon["acronym"]==acronym]

In [20]:
for record in alex.to_dict('records'):
    print(record)

{'Unnamed: 0': 351, 'acronym': 'CRR', 'summary': 'Monetary policy is the process by which the monetary authority of a country, generally  central bank, controls the supply of money in the economy by its control over interest rates in order to maintain price stability and achieve high economic growth. In India, the central monetary authority is the Reserve Bank of India (RBI).\nIt is designed to maintain the price stability in the economy. Other objectives of the monetary policy of India, as stated by RBI, are:\n\nPrice stability\nPrice stability implies promoting economic development with considerable emphasis on price stability. The centre of focus is to facilitate the environment which is favourable to the architecture that enables the developmental projects to run swiftly while also maintaining reasonable price stability.Controlled expansion of bank credit\nOne of the important functions of RBI is the controlled expansion of bank credit and money supply with special attention to sea